In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import copy

from itertools import product
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.naive_bayes import GaussianNB # Import Naive Bayes Classifier
from sklearn.svm import SVC, LinearSVC # Import SVM Classifier
from sklearn.neighbors import KNeighborsClassifier # Import KNN Classifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import tree
from pprint import pprint
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold
from prettytable import PrettyTable
from scipy import stats
from matplotlib.pyplot import pie, axis, show
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_classif, SelectFromModel, RFE
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score

from helpers import Trainer, plot_conf_mat, plot_roc, plot_roc_avg, plot_pie_chart, stats_test

random.seed(0)

In [2]:
X_cs_15 = np.load('npy_data/X_combinedSampling_15Features_task2.npy')
y_cs_15 = np.load('npy_data/y_combinedSampling_15Features_task2.npy')
print('X_cs_15.shape, y_cs_15.shape', X_cs_15.shape, y_cs_15.shape)

classes = ['Female', 'Male']

FileNotFoundError: [Errno 2] No such file or directory: 'npy_data_2/X_combinedSampling_15Features_task2.npy'

In [ ]:
plot_pie_chart(classes, y_cs_15, 15)

In [ ]:
kfold = StratifiedKFold(10, random_state=False, shuffle=True)

folds_15 = kfold.split(X_cs_15, y_cs_15)
folds_15 = [(train, test) for train, test in folds_15]

In [ ]:
data_classes_15  = []

for i in range(2):
    data_classes_15.append(len(y_cs_15[y_cs_15==i]))
    
cls2clr = {
    0: 'red',
    1: 'green'
}
clf2clr = {
    0: 'red',
    1: 'green',
    2: 'blue',
    3: 'purple',
    4: 'cornflowerblue',
}

In [ ]:
trainer = Trainer(X=X_cs_15, y=y_cs_15, folds=folds_15, data_classes=data_classes_15, classes=classes)

In [ ]:
tables, xgboost_metrics_15 = trainer.train_model(model_name='XGBoost')
table1, table2, conf_mat, conf_mat_norm = tables
print('----------------15 Selected Features----------------')
print(table1)
print(table2)
print(conf_mat)
plot_conf_mat(conf_mat_norm, '15 selected')
plot_roc(xgboost_metrics_15, data_classes_15, cls2clr, 'XGBoost')

In [ ]:
tables, adaboost_metrics_15 = trainer.train_model(model_name='AdaBoost')
table1, table2, conf_mat, conf_mat_norm = tables
print('----------------15 Selected Features----------------')
print(table1)
print(table2)
print(conf_mat)
plot_conf_mat(conf_mat_norm, '15 selected')
plot_roc(adaboost_metrics_15, data_classes_15, cls2clr, 'AdaBoost')

In [ ]:
tables, xtrees_metrics_15 = trainer.train_model(model_name='XTrees')
table1, table2, conf_mat, conf_mat_norm = tables
print('----------------15 Selected Features----------------')
print(table1)
print(table2)
print(conf_mat)
plot_conf_mat(conf_mat_norm, '15 selected')
plot_roc(xtrees_metrics_15, data_classes_15, cls2clr, 'XTrees')

In [ ]:
tables, knn_metrics_15 = trainer.train_model(model_name='KNN')
table1, table2, conf_mat, conf_mat_norm = tables
print('----------------15 Selected Features----------------')
print(table1)
print(table2)
print(conf_mat)
plot_conf_mat(conf_mat_norm, '15 selected')
plot_roc(knn_metrics_15, data_classes_15, cls2clr, 'KNN')

In [ ]:
tables, dt_metrics_15 = trainer.train_model(model_name='DT')
table1, table2, conf_mat, conf_mat_norm = tables
print('----------------15 Selected Features----------------')
print(table1)
print(table2)
print(conf_mat)
plot_conf_mat(conf_mat_norm, '15 selected')
plot_roc(dt_metrics_15, data_classes_15, cls2clr, 'DT')

In [ ]:
plot_roc_avg([xgboost_metrics_15,
                adaboost_metrics_15,
                xtrees_metrics_15,
                knn_metrics_15,
                dt_metrics_15], 
               ['XGBoost',
                'AdaBoost',
                'XTrees',
                'KNN',
                'DT'], 
               clf2clr,
            micro=True, macro=False)

In [ ]:
plot_roc_avg([xgboost_metrics_15,
                adaboost_metrics_15,
                xtrees_metrics_15,
                knn_metrics_15,
                dt_metrics_15], 
               ['XGBoost',
                'AdaBoost',
                'XTrees',
                'KNN',
                'DT'], 
               clf2clr,
               micro=False, macro=True)

In [ ]:
all_scores = [xgboost_metrics_15['recall'], 
              adaboost_metrics_15['recall'], 
              xtrees_metrics_15['recall'], 
              knn_metrics_15['recall'], 
              dt_metrics_15['recall']]
all_clf = ['XGBoost',
           'AdaBoost',
           'XTrees',
           'KNN',
           'DT']
print(stats_test(all_scores, all_clf))